SAMRefiner
=====

**SAMRefiner: Taming Segment Anything Model for Universal Mask Refinement**

 * Paper: https://arxiv.org/abs/2502.06756

![SAMRefiner Overview](../assets/samrefiner_overview.jpg)

 * Installation

```bash
git clone https://github.com/linyq2117/SAMRefiner.git SAMRefiner_repo

# create conda env
conda create -n SAMRefiner python=3.8 -y
conda activate SAMRefiner

# install packages
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 --extra-index-url https://download.pytorch.org/whl/cu113
pip install opencv-python tqdm matplotlib scipy
pip install FastGeodis --no-build-isolation --no-cache-dir

cd SAMRefiner_repo/segment-anything; pip install -e .; cd ../../
```